In [1]:
import pandas as pd

In [2]:
# --- Section 1: GILT MARKET (Gilt Yields) Data Processing ---

gilt_filename = r'data\20250803 - Historical Average Daily Conventional Gilt Yields.csv'

try:
    # Read the CSV, skipping the first two rows (metadata/header info)
    df_gilts = pd.read_csv(
        gilt_filename,
        skiprows=2
    )

    # Drop any rows that are entirely empty (e.g., if there were extra blank lines)
    df_gilts = df_gilts.dropna(how='all')

    # Rename columns for clarity
    df_gilts = df_gilts.rename(columns={
        df_gilts.columns[0]: 'Month',
        df_gilts.columns[1]: 'Short',
        df_gilts.columns[2]: 'Medium',
        df_gilts.columns[3]: 'Long',
        df_gilts.columns[4]: 'Ultra-Long'
    })

    # Strip any leading/trailing whitespace (including non-breaking spaces like '\xa0') from the 'Month' column
    df_gilts['Month'] = df_gilts['Month'].astype(str).str.strip()

    # Convert the 'Month' column to datetime objects.
    # We add MonthEnd(0) to set the date to the last day of the month for consistency.
    df_gilts['Month'] = pd.to_datetime(df_gilts['Month'], format='%b-%Y', errors='coerce') + pd.offsets.MonthEnd(0)

    # Add this line to drop rows where Month conversion failed (NaT index) for robustness
    df_gilts = df_gilts.dropna(subset=['Month'])

    # Set the 'Month' column as the DataFrame index
    df_gilts = df_gilts.set_index('Month')

    # Clean column names by removing any leading/trailing spaces
    df_gilts.columns = df_gilts.columns.str.strip()

    # Process the yield columns: remove '%' sign and convert to float
    yield_cols = ['Short', 'Medium', 'Long', 'Ultra-Long']

    # Remove rows where any yield column contains non-numeric values (excluding NaN)
    # First, process the values by removing '%'
    for col in yield_cols:
        if col in df_gilts.columns:
            # Convert to string first to handle potential NaNs or non-string types, then replace
            df_gilts[col] = df_gilts[col].astype(str).str.replace('%', '', regex=False)

    # Keep only rows where all yield columns are either numeric or NaN after '%' removal
    # Define the helper function (if not already defined elsewhere)
    def is_float_or_nan(x):
        try:
            float(x)
            return True
        except ValueError:
            return pd.isna(x) or (isinstance(x, str) and x.strip() == '')

    # Apply the mask. Suppress FutureWarning for applymap, or use .map() if you prefer.
    # You might consider .map() for Series if processing column by column or if performance is critical.
    # For a DataFrame with multiple columns, applymap or apply with lambda and Series.map is typical.
    mask = df_gilts[yield_cols].applymap(is_float_or_nan).all(axis=1)
    df_gilts = df_gilts[mask]

    # Now convert columns to float
    for col in yield_cols:
        if col in df_gilts.columns:
            df_gilts[col] = df_gilts[col].astype(float)
        # Rename columns for clarity, e.g., 'Short' becomes 'Gilt_Short_Yield'
        df_gilts = df_gilts.rename(columns={col: f'Gilt_{col}_Yield'})

    print("--- Gilt Yields Data (df_gilts) ---")
    print(df_gilts.head())
    print("\nDataFrame Info:")
    df_gilts.info()
    print("\n" + "="*50 + "\n")

except FileNotFoundError:
    print(f"Error: Gilt Yields file '{gilt_filename}' not found. Skipping Gilt Yields processing.")
    df_gilts = pd.DataFrame() # Create an empty DataFrame to prevent errors later
except Exception as e:
    print(f"An unexpected error occurred during Gilt Yields processing: {e}")
    df_gilts = pd.DataFrame() # Create an empty DataFrame on other errors


--- Gilt Yields Data (df_gilts) ---
            Gilt_Short_Yield  Gilt_Medium_Yield  Gilt_Long_Yield  \
Month                                                              
1998-04-30              5.91               5.70             5.71   
1998-05-31              5.82               5.57             5.55   
1998-06-30              6.17               5.64             5.43   
1998-07-31              6.06               5.57             5.38   
1998-08-31              5.52               5.19             5.11   

            Gilt_Ultra-Long_Yield  
Month                              
1998-04-30                    NaN  
1998-05-31                    NaN  
1998-06-30                    NaN  
1998-07-31                    NaN  
1998-08-31                    NaN  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 328 entries, 1998-04-30 to 2025-07-31
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 -------------- 

C:\Users\epoch_bpjmdqk\AppData\Local\Temp\ipykernel_11196\4174078042.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df_gilts[yield_cols].applymap(is_float_or_nan).all(axis=1)


In [12]:
# --- Section 2: Bank Rate Data Processing ---

bank_rate_filename = r'Hypoth\data\Bank Rate history and data  Bank of England Database.csv'

try:
    df_bank_rate = pd.read_csv(bank_rate_filename)

    # Drop any rows that are entirely empty
    df_bank_rate = df_bank_rate.dropna(how='all')

    if df_bank_rate.empty:
        raise ValueError("Bank Rate CSV loaded but is empty. Check the file content.")

    # Convert 'Date Changed' to datetime objects using the specified format
    # Use errors='coerce' to turn unparseable dates into NaT
    df_bank_rate['Date Changed'] = pd.to_datetime(df_bank_rate['Date Changed'], format='%d %b %y', errors='coerce')

    # Drop rows where date conversion failed (i.e., Date Changed is NaT)
    df_bank_rate = df_bank_rate.dropna(subset=['Date Changed'])

    # Set 'Date Changed' as the index and sort it in ascending order (chronological)
    df_bank_rate = df_bank_rate.set_index('Date Changed').sort_index()

    # Rename the 'Rate' column for clarity
    df_bank_rate = df_bank_rate.rename(columns={'Rate': 'Bank_Rate'})

    # Convert the 'Bank_Rate' column to float
    df_bank_rate['Bank_Rate'] = df_bank_rate['Bank_Rate'].astype(float)

    print("--- Bank Rate Data (df_bank_rate) ---")
    print(df_bank_rate.head())
    print("\nDataFrame Info:")
    df_bank_rate.info()
    print("\n" + "="*50 + "\n")

except FileNotFoundError:
    print(f"Error: Bank Rate file '{bank_rate_filename}' not found. Skipping Bank Rate processing.")
    df_bank_rate = pd.DataFrame() # Create an empty DataFrame to prevent errors later
except Exception as e:
    print(f"An error occurred during Bank Rate processing: {e}")
    df_bank_rate = pd.DataFrame()


Error: Bank Rate file 'Hypoth\data\Bank Rate history and data  Bank of England Database.csv' not found. Skipping Bank Rate processing.


In [4]:
# --- Section 3: SONIA Data Processing (REVISED BASED ON YOUR LATEST INPUT) ---

sonia_filename = r'data\SONIA.csv'

try:
    # Read the CSV. Explicitly specify the separator as comma.
    # The header is on the first line, so no skiprows is needed.
    df_sonia = pd.read_csv(sonia_filename, sep=',')

    # Drop any rows that are entirely empty
    df_sonia = df_sonia.dropna(how='all')

    # The actual SONIA rate column has a very long, messy name.
    # It should be the second column (index 1) after the 'Date' column.
    if len(df_sonia.columns) >= 2:
        long_sonia_col_name = df_sonia.columns[1] # Get the second column name
    else:
        raise ValueError("SONIA CSV does not have enough columns after parsing with comma delimiter. "
                         "Check if the file is truly comma-separated.")

    # Rename the columns for clarity
    df_sonia = df_sonia.rename(columns={
        df_sonia.columns[0]: 'Date', # First column is 'Date'
        long_sonia_col_name: 'SONIA_Rate' # Rename the long column
    })

    # Convert 'Date' to datetime objects using the specified format
    df_sonia['Date'] = pd.to_datetime(df_sonia['Date'], format='%d %b %y', errors='coerce')

    # Drop rows where date conversion failed (NaT)
    df_sonia = df_sonia.dropna(subset=['Date'])

    # Set 'Date' as the index and sort it in ascending order (chronological)
    df_sonia = df_sonia.set_index('Date').sort_index()

    # Convert the 'SONIA_Rate' column to float
    df_sonia['SONIA_Rate'] = df_sonia['SONIA_Rate'].astype(float)

    print("--- SONIA Data (df_sonia) ---")
    print(df_sonia.head())
    print("\nDataFrame Info:")
    df_sonia.info()
    print("\n" + "="*50 + "\n")

except FileNotFoundError:
    print(f"Error: SONIA file '{sonia_filename}' not found. Skipping SONIA processing.")
    df_sonia = pd.DataFrame()
except Exception as e:
    print(f"An error occurred during SONIA processing: {e}")
    df_sonia = pd.DataFrame()

--- SONIA Data (df_sonia) ---
            SONIA_Rate
Date                  
2014-01-02      0.4264
2014-01-03      0.4256
2014-01-06      0.4304
2014-01-07      0.4332
2014-01-08      0.4282

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2925 entries, 2014-01-02 to 2025-07-30
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SONIA_Rate  2925 non-null   float64
dtypes: float64(1)
memory usage: 45.7 KB




In [5]:
# --- Section 4: Stock Data Processing ---

stock_filename = r'data\uk_stock_data.csv'

try:
    df_stocks = pd.read_csv(stock_filename)

    # Drop any rows that are entirely empty
    df_stocks = df_stocks.dropna(how='all')

    # Convert 'Date' to datetime objects. Pandas is usually good at inferring YYYY-MM-DD.
    df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], errors='coerce')

    # Drop rows with failed date parsing
    df_stocks = df_stocks.dropna(subset=['Date'])

    # Set 'Date' as the index and sort it
    df_stocks = df_stocks.set_index('Date').sort_index()

    # Convert all stock price columns to numeric, coercing any errors (like non-numeric strings) to NaN
    # We apply this to all columns that are not the index.
    for col in df_stocks.columns:
        df_stocks[col] = pd.to_numeric(df_stocks[col], errors='coerce')

    print("--- Stock Data (df_stocks) ---")
    print(df_stocks.head())
    print("\nDataFrame Info:")
    df_stocks.info()
    print("\n" + "="*50 + "\n")

except FileNotFoundError:
    print(f"Error: Stock data file '{stock_filename}' not found. Skipping Stock processing.")
    df_stocks = pd.DataFrame()
except Exception as e:
    print(f"An error occurred during Stock processing: {e}")
    df_stocks = pd.DataFrame()

--- Stock Data (df_stocks) ---
                  ^FTSE      LLOY.L      BARC.L        AV.L       PSN.L  \
Date                                                                      
2000-01-03          NaN  498.020142  408.222687  972.562134  237.982162   
2000-01-04  6665.899902  466.833557  382.565521  908.244202  237.982162   
2000-01-05  6535.899902  466.190460  373.402405  880.958130  234.024033   
2000-01-06  6447.200195  450.757965  359.428284  873.161926  234.024033   
2000-01-07  6504.799805  435.003906  351.410492  892.652161  235.013565   

                LAND.L  
Date                    
2000-01-03  650.739258  
2000-01-04  631.986145  
2000-01-05  646.519775  
2000-01-06  646.519775  
2000-01-07  649.333130  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6364 entries, 2000-01-03 to 2024-12-30
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ^FTSE   6313 non-null   float64
 1   LLOY.L  636

In [ ]:
# Merge all dataframes on their datetime index (outer join to keep all available data)
# Start by merging df_stocks and df_gilts (monthly), then merge df_sonia (daily)

# First, resample df_gilts to daily frequency to match others (forward fill monthly values)
df_gilts_daily = df_gilts.resample('D').ffill()

# Merge stocks and gilts
df_merged = df_stocks.join(df_gilts_daily, how='outer')

# Merge SONIA
df_merged = df_merged.join(df_sonia, how='outer')

# Merge Bank Rate 
df_merged = df_merged.join(df_bank_rate, how='outer')

print(df_merged.head())
print("\nMerged DataFrame Info:")
df_merged.info()

            ^FTSE  LLOY.L  BARC.L  AV.L  PSN.L  LAND.L  Gilt_Short_Yield  \
1998-04-30    NaN     NaN     NaN   NaN    NaN     NaN              5.91   
1998-05-01    NaN     NaN     NaN   NaN    NaN     NaN              5.91   
1998-05-02    NaN     NaN     NaN   NaN    NaN     NaN              5.91   
1998-05-03    NaN     NaN     NaN   NaN    NaN     NaN              5.91   
1998-05-04    NaN     NaN     NaN   NaN    NaN     NaN              5.91   

            Gilt_Medium_Yield  Gilt_Long_Yield  Gilt_Ultra-Long_Yield  \
1998-04-30                5.7             5.71                    NaN   
1998-05-01                5.7             5.71                    NaN   
1998-05-02                5.7             5.71                    NaN   
1998-05-03                5.7             5.71                    NaN   
1998-05-04                5.7             5.71                    NaN   

            SONIA_Rate  
1998-04-30         NaN  
1998-05-01         NaN  
1998-05-02         NaN  
1998